In [1]:
import pandas as pd
df = pd.read_excel('RD3.xlsx')
df.head()

,Listing_Title,City,Area,Location,Zone,Propery_Type,Size_In_Sqft,Carpet_Area_Sqft,Bedrooms,Bathrooms,...,Property_Age,Brokerage,Floor_No,Total_floors_In_Building,Maintenance_Charge,Nearby_Facilities,Recomened for,Muncipla Water Or Bore Water,Type of Society,Road Connectivity
0,1 BHK Flat for Rent,Nagpur,shatabdisquare,"Shatabdi Square, Nagpur, Maharashtra",south zone,1 BHK Flat,800 .sqft,NaN,1,1,...,Less than 5 years,No,2 Floor,2 Floors,No,"school,hospital",family,Borewell,Non-Gated,under 2 km
1,1 BHK Flat For Rent,Nagpur,manewada,"Manewada, Nagpur, Maharashtra",south zone,1 BHK Flat,700 .sqft,NaN,1,1,...,new Construction,No,2 Floor,3 Floors,No,"school,hospital",family,Both,Non-Gated,under 1 km
2,1 RK Flat for Rent,Nagpur,hudkeswar,Devashree Krishnam Nagari Location Hudkeshwar ...,south zone,1 RK,1000.sqft,NaN,1,1,...,NaN,No,NaN,NaN,No,"School,Hospital, metro station, mall",family,Borewell,Gated,under 2 km
3,2 BHK Flat for Rent,Nagpur,manish nagar,"Nandita Apartment manish nagar , New Manish Na...",south zone,2 BHK Flat,500 .sqft,NaN,2,2,...,NaN,No,NaN,NaN,No,"school, hospital, metro station, airport",NaN,Borewell,Non-Gated,under 2 km
4,1 BHK Flat for Rent,Nagpur,beltarodi,"SDPL Aashray Beltarodi II, Beltarodi, Nagpur",south zone,1 BHK Flat,525.sqft,NaN,1,1,...,NaN,No,NaN,NaN,No,"school, hospital, metro station, airport",NaN,Municipal,Gated,under 1 km


In [3]:
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt  
import seaborn as sns  
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import optuna
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')  # ignore notifications
import sys
import os

In [4]:
df['Frurnishing_Status'].unique()
mapping = {
    'Semi Furnished': 'Semi Furnished',
    'Semi-Furnished': 'Semi Furnished',
    'Furnished': 'Fully Furnished',
    'Unfurnished': 'Unfurnished',
    'Fully Furnished': 'Fully Furnished'
}
df['Frurnishing_Status'] = df['Frurnishing_Status'].map(mapping)
df['Frurnishing_Status'].value_counts()

Frurnishing_Status
Semi Furnished     390
Fully Furnished    320
Unfurnished        290
Name: count, dtype: int64

In [5]:
df['Recomened for'].unique()
mapping = {
    'family': 'Family',
    'Family': 'Family',
    'bachelor': 'Bachelors',
    'batchelor': 'Bachelors',
    'batchlor': 'Bachelors',
    'Bachelors': 'Bachelors',
    'family,company': 'Family and Company',
    'family,batchelor': 'Family and Bachelors',
    'family, batchelor': 'Family and Bachelors',
    'Anyone': 'Anyone',
    '-1': 'Anyone',
    'family,batchelor,company': 'Anyone',
}

df['Recomened for'] = df['Recomened for'].map(mapping)
df['Recomened for'].value_counts()


Recomened for
Anyone                  336
Family                  285
Bachelors               218
Family and Company       36
Family and Bachelors     30
Name: count, dtype: int64

In [6]:
df['Amenities'].unique()
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Step 1: Replace -1 and NaN with empty string
df['Amenities'] = df['Amenities'].replace(-1, '').fillna('')

# Step 2: Convert to lowercase, remove extra spaces, and correct typos
df['Amenities'] = df['Amenities'].astype(str).str.lower()
df['Amenities'] = df['Amenities'].str.replace('geted', 'gated')
df['Amenities'] = df['Amenities'].str.replace('cummunity', 'community')
df['Amenities'] = df['Amenities'].str.replace('backpu', 'backup')
df['Amenities'] = df['Amenities'].str.replace('allowd', 'allowed')
df['Amenities'] = df['Amenities'].str.replace('alloed', 'allowed')
df['Amenities'] = df['Amenities'].str.replace('paower', 'power')
df['Amenities'] = df['Amenities'].str.replace('intercome', 'intercom')
df['Amenities'] = df['Amenities'].str.replace('sporty', 'sports')
df['Amenities'] = df['Amenities'].str.replace('sysytem', 'system')
df['Amenities'] = df['Amenities'].str.replace(' ,', ',')  # remove space before commas

# Step 3: Remove extra spaces around commas and values
df['Amenities'] = df['Amenities'].str.replace(r'\s*,\s*', ',', regex=True)
df['Amenities'] = df['Amenities'].str.strip()

# Step 4: Convert comma-separated strings to list
df['Amenities'] = df['Amenities'].apply(lambda x: x.split(',') if x else [])

# Step 5: Apply MultiLabelBinarizer
mlb = MultiLabelBinarizer()
amenities_encoded = pd.DataFrame(mlb.fit_transform(df['Amenities']), columns=mlb.classes_)

# Step 6: Concatenate with original dataframe
df = pd.concat([df.drop(columns=['Amenities']), amenities_encoded], axis=1)

# Optional: Check the result
print(df.head())
# Check for binary columns (containing only 0, 1, or NaN)
binary_columns = [col for col in df.columns if df[col].dropna().isin([0, 1]).all()]

print("Binary columns are:")
print(binary_columns)
# Define similar columns you want to merge
merge_map = {
    'gas_pipeline': ['gas pipeline', 'gas pipline'],
    'sports_facility': ['sport facility', 'sports facility','basket ball court'],
    'kids_area': ['kids area', 'kids arear', 'kids pool'],
    'power_backup': ['power backup', 'powert backup'],
    'Garden': ['close to jain temple', 'garden'],
    'Fire Support': ['fire fighting', 'fire fighting system'],
    'Parking': ['parking', 'paeking']
}

# Merge each group
for new_col, cols_to_merge in merge_map.items():
    df[new_col] = df[cols_to_merge].max(axis=1)
    df.drop(columns=cols_to_merge, inplace=True)

# Print remaining binary columns
remaining_binary_cols = [col for col in df.columns if df[col].dropna().isin([0, 1]).all()]
print("Remaining binary columns:", remaining_binary_cols)


           Listing_Title    City            Area  \
0   1 BHK Flat for Rent   Nagpur  shatabdisquare   
1  1 BHK Flat  For Rent   Nagpur        manewada   
2     1 RK Flat for Rent  Nagpur       hudkeswar   
3    2 BHK Flat for Rent  Nagpur    manish nagar   
4    1 BHK Flat for Rent  Nagpur       beltarodi   

                                            Location        Zone Propery_Type  \
0               Shatabdi Square, Nagpur, Maharashtra  south zone   1 BHK Flat   
1                      Manewada, Nagpur, Maharashtra  south zone   1 BHK Flat   
2  Devashree Krishnam Nagari Location Hudkeshwar ...  south zone         1 RK   
3  Nandita Apartment manish nagar , New Manish Na...  south zone   2 BHK Flat   
4       SDPL Aashray Beltarodi II, Beltarodi, Nagpur  south zone   1 BHK Flat   

  Size_In_Sqft Carpet_Area_Sqft  Bedrooms  Bathrooms  ...  parking  \
0    800 .sqft              NaN         1          1  ...        0   
1    700 .sqft              NaN         1          1  ...   

In [7]:
df['Nearby_Facilities'].unique()
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Step 1: Replace -1 and NaN with empty strings
df['Nearby_Facilities'] = df['Nearby_Facilities'].replace(-1, '').fillna('')

# Step 2: Convert to lowercase and fix typos (as needed)
df['Nearby_Facilities'] = df['Nearby_Facilities'].astype(str).str.lower()
df['Nearby_Facilities'] = df['Nearby_Facilities'].str.replace('hospit[l|a]', 'hospital', regex=True)
df['Nearby_Facilities'] = df['Nearby_Facilities'].str.replace('schooll', 'school')
df['Nearby_Facilities'] = df['Nearby_Facilities'].str.replace(' ,', ',')  # remove space before commas

# Step 3: Remove extra spaces around commas and values
df['Nearby_Facilities'] = df['Nearby_Facilities'].str.replace(r'\s*,\s*', ',', regex=True)
df['Nearby_Facilities'] = df['Nearby_Facilities'].str.strip()

# Step 4: Convert comma-separated strings to list
df['Nearby_Facilities'] = df['Nearby_Facilities'].apply(lambda x: x.split(',') if x else [])

# Step 5: Apply MultiLabelBinarizer
mlb_nf = MultiLabelBinarizer()
nearby_encoded = pd.DataFrame(mlb_nf.fit_transform(df['Nearby_Facilities']), columns=mlb_nf.classes_)

# Step 6: Concatenate with original dataframe
df = pd.concat([df.drop(columns=['Nearby_Facilities']), nearby_encoded], axis=1)

# Optional: Check result
print(df.head())
# Print remaining binary columns
remaining_binary_cols = [col for col in df.columns if df[col].dropna().isin([0, 1]).all()]
print("Remaining binary columns:", remaining_binary_cols)
# Define similar columns you want to merge
merge_map = {
    'ATM_Near_me': ['atm'],
    'Airport_Near_me': ['airport'],
    'Bus_Stop__Near_me': ['bus stop'],
    'Hospital_Near_me': ['hospital','hospitall'],
    'Mall_Near_me': ['mall'],
    'Market_Near_me': ['market'],
    'Metro_Station_Near_me': ['metro station'],
    'Park_Near_me': ['park'],
    'School_Near_me': ['school'],
}

# Merge each group
for new_col, cols_to_merge in merge_map.items():
    df[new_col] = df[cols_to_merge].max(axis=1)
    df.drop(columns=cols_to_merge, inplace=True)

# Print remaining binary columns
remaining_binary_cols = [col for col in df.columns if df[col].dropna().isin([0, 1]).all()]
print("Remaining binary columns:", remaining_binary_cols)


           Listing_Title    City            Area  \
0   1 BHK Flat for Rent   Nagpur  shatabdisquare   
1  1 BHK Flat  For Rent   Nagpur        manewada   
2     1 RK Flat for Rent  Nagpur       hudkeswar   
3    2 BHK Flat for Rent  Nagpur    manish nagar   
4    1 BHK Flat for Rent  Nagpur       beltarodi   

                                            Location        Zone Propery_Type  \
0               Shatabdi Square, Nagpur, Maharashtra  south zone   1 BHK Flat   
1                      Manewada, Nagpur, Maharashtra  south zone   1 BHK Flat   
2  Devashree Krishnam Nagari Location Hudkeshwar ...  south zone         1 RK   
3  Nandita Apartment manish nagar , New Manish Na...  south zone   2 BHK Flat   
4       SDPL Aashray Beltarodi II, Beltarodi, Nagpur  south zone   1 BHK Flat   

  Size_In_Sqft Carpet_Area_Sqft  Bedrooms  Bathrooms  ...  Parking airport  \
0    800 .sqft              NaN         1          1  ...        0       0   
1    700 .sqft              NaN         1   

In [8]:
print(df['Size_In_Sqft'])
df['Size_In_Sqft'] = df['Size_In_Sqft'].str.replace(r'\.?\s*sqft', '', regex=True).str.strip()
print(df['Size_In_Sqft'])

0      800 .sqft
1      700 .sqft
2      1000.sqft
3      500 .sqft
4       525.sqft
         ...    
995     900 sqft
996     600 sqft
997    1100 sqft
998     900 sqft
999     800.sqft
Name: Size_In_Sqft, Length: 1000, dtype: object
0       800
1       700
2      1000
3       500
4       525
       ... 
995     900
996     600
997    1100
998     900
999     800
Name: Size_In_Sqft, Length: 1000, dtype: object


In [9]:
print(df['Carpet_Area_Sqft'])
df['Carpet_Area_Sqft'] = df['Carpet_Area_Sqft'].str.replace(r'\.?\s*sqft', '', regex=True).str.strip()
print(df['Carpet_Area_Sqft'])

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
         ...    
995    1100 sqft
996     750 sqft
997     900 sqft
998    1100 sqft
999     750.sqft
Name: Carpet_Area_Sqft, Length: 1000, dtype: object
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
995    1100
996     750
997     900
998    1100
999     750
Name: Carpet_Area_Sqft, Length: 1000, dtype: object


In [10]:
import re
print(df['Total_floors_In_Building'])
df['Total_floors_In_Building'] = df['Total_floors_In_Building'].str.replace(r'\.?\s*floors?', '', regex=True, flags=re.IGNORECASE).str.strip()
print(df['Total_floors_In_Building'])



0      2 Floors
1      3 Floors
2           NaN
3           NaN
4           NaN
         ...   
995    9 Floors
996    5 Floors
997    3 Floors
998    7 Floors
999    5 Floors
Name: Total_floors_In_Building, Length: 1000, dtype: object
0        2
1        3
2      NaN
3      NaN
4      NaN
      ... 
995      9
996      5
997      3
998      7
999      5
Name: Total_floors_In_Building, Length: 1000, dtype: object


In [11]:
import re
print(df['Floor_No'])
df['Floor_No'] = df['Floor_No'].str.replace(r'\.?\s*floors?', '', regex=True, flags=re.IGNORECASE).str.strip()
df['Floor_No'] = df['Floor_No'].str.replace(r'Ground', 'G', regex=True, flags=re.IGNORECASE)
print(df['Floor_No'])


0      2 Floor
1      2 Floor
2          NaN
3          NaN
4          NaN
        ...   
995    1 Floor
996    2 Floor
997    2 Floor
998    2 Floor
999    3 Floor
Name: Floor_No, Length: 1000, dtype: object
0        2
1        2
2      NaN
3      NaN
4      NaN
      ... 
995      1
996      2
997      2
998      2
999      3
Name: Floor_No, Length: 1000, dtype: object


In [12]:
df['Listing_Title'].unique()
mapping = {
    '1 BHK Flat for Rent ':'1 BHK Flat for Rent',
    '1 BHK Flat  For Rent ':'1 BHK Flat for Rent',
    '1 BHK Flat for Rent':'1 BHK Flat for Rent',
    '1 BHK Flat\xa0for Rent':'1 BHK Flat for Rent',
    
    '3 BHK Flat for rent':'3 BHK Flat for rent',
    '3 BHK Apartment for Rent':'3 BHK Flat for rent',
    '3 BHK Flat\xa0for Rent':'3 BHK Flat for rent',
    
    '2 BHK Flat for Rent':'2 BHK Flat for Rent',
    '2 BHK  Flat for Rent':'2 BHK Flat for Rent',
    '2 BHK Flat for Rent ':'2 BHK Flat for Rent',
    '2 BHK Flat\xa0for Rent':'2 BHK Flat for Rent',

    '4 BHK Flat for rent':'4 BHK Flat for rent',

    '1 RK Flat for Rent':'1 RK Flat for Rent',
    'Studio Apartment for Rent':'Studio Apartment for Rent',
    
    '1 BHK Independent House\xa0for Rent':'1 BHK Independent House for Rent',
    '1 BHK Independent House for Rent':'1 BHK Independent House for Rent',
    
    '3 BHK Independent House\xa0for Rent':'3 BHK Independent House for Rent',
    
    '4 BHK Independent House\xa0for Rent':'4 BHK Independent House for Rent',
    
    '5 BHK Independent House\xa0for Rent':'5 BHK Independent House for Rent',

    '1 BHK Independent Builder Floor\xa0for Rent':'1 BHK Independent Builder Floor for Rent',
    
    '2 BHK Independent Builder Floor for Rent':'2 BHK Independent Builder Floor for Rent',
    '2 BHK Independent Builder Floor\xa0for Rent':'2 BHK Independent Builder Floor for Rent',
    
    '3 BHK Independent Builder Floor for Rent':'3 BHK Independent Builder Floor for Rent',
    
    '4 BHK Independent Builder Floor\xa0for Rent':'4 BHK Independent Builder Floor for Rent',

    '1 BHK Villa\xa0for Rent':'1 BHK Villa for Rent',
    
    '3 BHK Villa\xa0for Rent':'3 BHK Villa for rent',
    '3 BHK villa for rent':'3 BHK Villa for rent',
    
    '4 BHK Villa\xa0for Rent':'4 BHK Villa for Rent',

    '1 RK Independent House\xa0for Rent':'1 RK Independent House for Rent',
    '1 RK Independent house\xa0for Rent':'1 RK Independent House for Rent',

    '3 BHK Duplex\xa0for Rent':'3 BHK Duplex for Rent',

    
}

df['Listing_Title'] = df['Listing_Title'].map(mapping)
df['Listing_Title'].value_counts()


Listing_Title
2 BHK Flat for Rent                         231
Studio Apartment for Rent                   218
3 BHK Flat for rent                         208
1 RK Flat for Rent                          183
3 BHK Independent House for Rent             31
1 BHK Flat for Rent                          14
1 BHK Independent House for Rent             12
1 RK Independent House for Rent              12
2 BHK Independent Builder Floor for Rent     10
1 BHK Independent Builder Floor for Rent      7
3 BHK Villa for rent                          6
3 BHK Independent Builder Floor for Rent      3
4 BHK Independent Builder Floor for Rent      2
4 BHK Villa for Rent                          2
1 BHK Villa for Rent                          2
3 BHK Duplex for Rent                         2
5 BHK Independent House for Rent              2
4 BHK Independent House for Rent              2
4 BHK Flat for rent                           1
Name: count, dtype: int64

In [13]:
df['Zone'].unique()
mapping = {
    'south zone':'South Zone',
    'east zone':'East Zone',
    'west zone':'West Zone',
    'West':'West Zone',
    'north zone':'North Zone',
    'rural':'rural',
}
df['Zone'] = df['Zone'].map(mapping)
df['Zone'].value_counts()

Zone
South Zone    318
West Zone     257
East Zone     219
North Zone    204
rural           2
Name: count, dtype: int64

In [14]:

# Extract Room (BHK, RK, Studio)
df['Room'] = df['Listing_Title'].str.extract(r'(\d+\s*BHK|\d+\s*RK|Studio Apartment)', expand=False)

# Extract Type (Flat, Independent House, etc.)
df['Type'] = df['Listing_Title'].str.extract(
    r'(Flat|Independent House|Independent Builder Floor|Villa|Duplex|Studio Apartment)', 
    expand=False
)

df.drop(columns=['Listing_Title'], inplace=True)
df.drop(columns=['Propery_Type'], inplace=True)



In [15]:
df['Property_Age'].unique()


age_mapping = {
    'Less than 5 years': 5,
    'More than 20 years': 20,
    'New Property': 0,
    'new Construction': 0,
    ' ': np.nan  
}
df['Property_Age'] = df['Property_Age'].replace(age_mapping)
df['Property_Age'] = df['Property_Age'].apply(
    lambda x: int(re.sub(r'\D', '', str(x))) if re.search(r'\d', str(x)) else x
)
df['Property_Age'] = pd.to_numeric(df['Property_Age'], errors='coerce').fillna(0).astype(int)
df['Property_Age'].value_counts()

Property_Age
0     176
5     117
1      88
3      67
10     60
2      52
15     52
4      49
11     48
9      46
12     45
6      44
8      41
14     40
13     38
7      37
Name: count, dtype: int64

In [16]:
import re

def convert_to_km(value):
    if pd.isna(value):
        return None
    
    value = str(value).lower().strip()
    
    # Extract number
    num_match = re.search(r'(\d+)', value)
    if not num_match:
        return None
    num = float(num_match.group(1))
    
    # Check unit
    if 'm' in value and 'km' not in value:  # meters
        return num / 1000
    else:  # kilometers
        return num

df['Road Connectivity'] = df['Road Connectivity'].apply(convert_to_km)
df['Road Connectivity'].value_counts()

Road Connectivity
1.0    467
2.0    261
5.0    198
0.5     32
4.0     25
3.0     15
6.0      2
Name: count, dtype: int64

In [17]:
df['Brokerage'].unique()
mapping = {
    'No':'NO',
    'yes':'YES',
    'Yes':'YES'
}
df['Brokerage'] = df['Brokerage'].map(mapping)
df['Brokerage'].value_counts()

Brokerage
NO     546
YES    454
Name: count, dtype: int64

In [18]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 49 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   City                          1000 non-null   object 
 1   Area                          1000 non-null   object 
 2   Location                      1000 non-null   object 
 3   Zone                          1000 non-null   object 
 4   Size_In_Sqft                  1000 non-null   object 
 5   Carpet_Area_Sqft              772 non-null    object 
 6   Bedrooms                      1000 non-null   int64  
 7   Bathrooms                     1000 non-null   int64  
 8   Balcony                       852 non-null    float64
 9   Frurnishing_Status            1000 non-null   object 
 10  Number_Of_Amenities           1000 non-null   int64  
 11  Security_Deposite             1000 non-null   float64
 12  Rent_Price                    1000 non-null   float64
 13  Prop

In [19]:
df = df.drop('Location', axis=1)

In [20]:
df.to_excel("clean.xlsx", index=False)

In [24]:
df['Brokerage'].value_counts()


Brokerage
NO     546
YES    454
Name: count, dtype: int64

In [34]:
df['Type of Society'].value_counts()


Type of Society
Gated        527
Non-Gated    369
Township     104
Name: count, dtype: int64

In [40]:
df['Recomened for'].value_counts()


Recomened for
Anyone                  336
Family                  285
Bachelors               218
Family and Company       36
Family and Bachelors     30
Name: count, dtype: int64

In [41]:
df['Type'].value_counts()

Type
Flat                         637
Studio Apartment             218
Independent House             59
Independent Builder Floor     22
Villa                         10
Duplex                         2
Name: count, dtype: int64

In [42]:
df['Frurnishing_Status'].value_counts()

Frurnishing_Status
Semi Furnished     390
Fully Furnished    320
Unfurnished        290
Name: count, dtype: int64

In [44]:
df['Type of Society'].value_counts()

Type of Society
Gated        527
Non-Gated    369
Township     104
Name: count, dtype: int64

In [51]:
df['Muncipla Water Or Bore Water'].value_counts()

Muncipla Water Or Bore Water
Borewell     487
Both         283
Municipal    230
Name: count, dtype: int64

In [53]:
df['Room'].value_counts()

Room
3 BHK               250
2 BHK               241
Studio Apartment    218
1 RK                195
1 BHK                35
4 BHK                 7
5 BHK                 2
Name: count, dtype: int64

In [55]:
df['Zone'].value_counts()

Zone
South Zone    318
West Zone     257
East Zone     219
North Zone    204
rural           2
Name: count, dtype: int64

In [57]:
df['Area'].value_counts()

Area
manewada                    156
dharampeth                  138
sitabuldi                   125
sadar                       115
shatabdi square             110
mihan                        20
beltarodi                    18
nandanwan                    16
manish nagar                 12
dhantoli                     12
dighori                      12
narendra nagar extension     11
besa                         10
mahal                        10
Hingna                       10
hingna                       10
Trimurti Nagar                8
khamla                        8
ram nagar                     8
kotewada                      8
zingabai takli                8
gandhibagh                    8
pratap nagar                  8
jaitala                       8
rameshwari                    8
mankapur                      8
jaripatka                     8
itwari                        8
ganeshpeth                    8
hudkeswar                     6
kharbi                        6
wat

## find null value

In [60]:
null_counts = df.isnull().sum()
print(null_counts)

City                              0
Area                              0
Zone                              0
Size_In_Sqft                      0
Carpet_Area_Sqft                228
Bedrooms                          0
Bathrooms                         0
Balcony                         148
Frurnishing_Status                0
Number_Of_Amenities               0
Security_Deposite                 0
Rent_Price                        0
Property_Age                      0
Brokerage                         0
Floor_No                        264
Total_floors_In_Building        264
Maintenance_Charge                0
Recomened for                    95
Muncipla Water Or Bore Water      0
Type of Society                   0
Road Connectivity                 0
gated community                   0
gym                               0
intercom                          0
lift                              0
pet allowed                       0
pool                              0
security                    

In [62]:
df = df.replace(" ", 0).fillna(0)



In [64]:
null_counts = df.isnull().sum()
print(null_counts)

City                            0
Area                            0
Zone                            0
Size_In_Sqft                    0
Carpet_Area_Sqft                0
Bedrooms                        0
Bathrooms                       0
Balcony                         0
Frurnishing_Status              0
Number_Of_Amenities             0
Security_Deposite               0
Rent_Price                      0
Property_Age                    0
Brokerage                       0
Floor_No                        0
Total_floors_In_Building        0
Maintenance_Charge              0
Recomened for                   0
Muncipla Water Or Bore Water    0
Type of Society                 0
Road Connectivity               0
gated community                 0
gym                             0
intercom                        0
lift                            0
pet allowed                     0
pool                            0
security                        0
water supply                    0
wifi          

## One-hot encode categorical features

In [67]:

print("--- Feature Engineering: One-Hot Encoding Categorical Features ---")
categorical_features = ['City','Area', 'Zone', 'Frurnishing_Status', 'Brokerage', 'Maintenance_Charge', 'Recomened for', 'Muncipla Water Or Bore Water', 'Type of Society', 'Room', 'Type']
for feature in categorical_features:
    if feature in df.columns:
        df = pd.get_dummies(df, columns=[feature], drop_first=True)
        print(f"One-hot encoded '{feature}'.")
    else:
        print(f"Warning: Categorical feature '{feature}' not found in DataFrame for encoding.")


# Transform 'Rent' using natural logarithm to reduce skewness
print("--- Feature Engineering: Log Transformation of 'Rent' ---")
# Add a small constant to Rent to avoid log(0) if any rent value is 0
df['Rent_Price'] = np.log1p(df['Rent_Price']) # Using log1p which is log(1+x)
print("Applied log transformation to 'Rent' column.")

print("\n--- DataFrame after Feature Engineering ---")
print(df.head())
print(df.info())


--- Feature Engineering: One-Hot Encoding Categorical Features ---
One-hot encoded 'City'.
One-hot encoded 'Area'.
One-hot encoded 'Zone'.
One-hot encoded 'Frurnishing_Status'.
One-hot encoded 'Brokerage'.
One-hot encoded 'Maintenance_Charge'.
One-hot encoded 'Recomened for'.
One-hot encoded 'Muncipla Water Or Bore Water'.
One-hot encoded 'Type of Society'.
One-hot encoded 'Room'.
One-hot encoded 'Type'.
--- Feature Engineering: Log Transformation of 'Rent' ---
Applied log transformation to 'Rent' column.

--- DataFrame after Feature Engineering ---
  Size_In_Sqft Carpet_Area_Sqft  Bedrooms  Bathrooms  Balcony  \
0          800                0         1          1      0.0   
1          700                0         1          1      0.0   
2         1000                0         1          1      0.0   
3          500                0         2          2      0.0   
4          525                0         1          1      0.0   

   Number_Of_Amenities  Security_Deposite  Rent_Price

In [69]:
# Define features (X) and target (y)
X = df.drop('Rent_Price', axis=1)
y = df['Rent_Price']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"\nData split into training (80%) and testing (20%) sets.")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")



Data split into training (80%) and testing (20%) sets.
X_train shape: (800, 124), y_train shape: (800,)
X_test shape: (200, 124), y_test shape: (200,)


In [71]:
numerical_cols = ['Size_In_Sqft', 'Carpet_Area_Sqft', 'Bedrooms', 'Bathrooms', 'Balcony',
'Number_Of_Amenities','Security_Deposite', 'Floor_No', 'Total_floors_In_Building',
'Road_Connectivity', 'gated_community', 'gym', 'intercom', 'lift', 'pet_allowed', 'pool',
'security', 'water_supply', 'wifi', 'gas_pipeline', 'sports_facility', 'kids_area',
'power_backup', 'Garden', 'Fire_Support', 'Parking', 'ATM_Near_me', 'Airport_Near_me',
'Bus_Stop__Near_me', 'Hospital_Near_me', 'Mall_Near_me', 'Market_Near_me',
'Metro_Station_Near_me', 'Park_Near_me', 'School_Near_me', 'Property_Age']
# Filter numerical_cols to only include columns actually present in X_train
numerical_cols_present = [col for col in numerical_cols if col in X_train.columns]

# 1. Replace empty strings with NaN
X_train = X_train.replace(r'^\s*$', np.nan, regex=True)
X_test = X_test.replace(r'^\s*$', np.nan, regex=True)

# 2. Convert all numeric columns to float
X_train[numerical_cols_present] = X_train[numerical_cols_present].apply(pd.to_numeric, errors='coerce')
X_test[numerical_cols_present] = X_test[numerical_cols_present].apply(pd.to_numeric, errors='coerce')

# 3. Fill NaN with median for each column
for col in numerical_cols_present:
    median_val = X_train[col].median()
    X_train[col].fillna(median_val, inplace=True)
    X_test[col].fillna(median_val, inplace=True)

# 4. Scale the numeric columns
scaler = StandardScaler()
X_train[numerical_cols_present] = scaler.fit_transform(X_train[numerical_cols_present])
X_test[numerical_cols_present] = scaler.transform(X_test[numerical_cols_present])


In [73]:
from sklearn.metrics import r2_score, mean_squared_error
print("\n--- Model Training: Random Forest Regressor ---")
# Initialize and train the Random Forest Regressor model
# Using parameters that generally give good results, but further tuning can be done.
rf_model = GradientBoostingRegressor(n_estimators=10000,learning_rate=0.0092,subsample=0.84555,max_depth=10,random_state=42)
rf_model.fit(X_train, y_train)
print("Random Forest Regressor model trained.")

# Make predictions on the test set
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
r2_rf = r2_score(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

print(f"\n--- Model Evaluation (Random Forest) ---")
print(f"R-squared (R^2) Score: {r2_rf:.4f}")
print(f"Mean Squared Error (MSE): {mse_rf:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_rf:.4f}")


--- Model Training: Random Forest Regressor ---
Random Forest Regressor model trained.

--- Model Evaluation (Random Forest) ---
R-squared (R^2) Score: 0.9997
Mean Squared Error (MSE): 0.0001
Root Mean Squared Error (RMSE): 0.0073


In [710]:
from sklearn.metrics import r2_score, mean_squared_error
print("\n--- Model Training: Random Forest Regressor ---")
# Initialize and train the Random Forest Regressor model
# Using parameters that generally give good results, but further tuning can be done.
rf_model = RandomForestRegressor(n_estimators=35000,n_jobs=-1,random_state=42)
rf_model.fit(X_train, y_train)
print("Random Forest Regressor model trained.")

# Make predictions on the test set
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
r2_rf = r2_score(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

print(f"\n--- Model Evaluation (Random Forest) ---")
print(f"R-squared (R^2) Score: {r2_rf:.4f}")
print(f"Mean Squared Error (MSE): {mse_rf:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_rf:.4f}")


--- Model Training: Random Forest Regressor ---
Random Forest Regressor model trained.

--- Model Evaluation (Random Forest) ---
R-squared (R^2) Score: 0.5031
Mean Squared Error (MSE): 0.1057
Root Mean Squared Error (RMSE): 0.3251


In [75]:
import pandas as pd
import numpy as np

# Assume you have your trained RandomForestRegressor model `rf_model`
# and your feature names list `feature_names` (the columns in your X_train after preprocessing)

# If you used a pipeline, get the feature names properly, e.g.:
# feature_names = ... (depends on your preprocessing step)

# Get feature importances (array of floats summing to 1)
importances = rf_model.feature_importances_
feature_names = X_train.columns
# Convert to percentage
importances_percentage = 100 * importances / importances.sum()

# Create a DataFrame to see importance by feature
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance (%)': importances_percentage
}).sort_values(by='Importance (%)', ascending=False)

print(feature_importance_df)

                     Feature  Importance (%)
6          Security_Deposite    9.982383e+01
8                   Floor_No    3.869617e-02
4                    Balcony    3.279915e-02
9   Total_floors_In_Building    2.718259e-02
76         Area_pratap nagar    2.271977e-02
..                       ...             ...
80                Area_sadar    7.685294e-11
84       Area_shatabdisquare    5.524987e-12
34              Park_Near_me    3.887374e-13
32            Market_Near_me    3.385364e-13
75        Area_parvati nagar    0.000000e+00

[124 rows x 2 columns]


In [749]:
from sklearn.metrics import r2_score, mean_squared_error
print("\n--- Model Training: Random Forest Regressor ---")
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
# Initialize and train the Random Forest Regressor model
# Using parameters that generally give good results, but further tuning can be done.
rf_model = GradientBoostingRegressor()
rf_model.fit(X_train, y_train)
print("Random Forest Regressor model trained.")

# Make predictions on the test set
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
r2_rf = r2_score(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

print(f"\n--- Model Evaluation (Random Forest) ---")
print(f"R-squared (R^2) Score: {r2_rf:.4f}")
print(f"Mean Squared Error (MSE): {mse_rf:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_rf:.4f}")



--- Model Training: Random Forest Regressor ---
Random Forest Regressor model trained.

--- Model Evaluation (Random Forest) ---
R-squared (R^2) Score: 0.4589
Mean Squared Error (MSE): 0.1151
Root Mean Squared Error (RMSE): 0.3393


In [77]:
# --- Saving the trained model and scaler ---
import joblib
print("\n--- Saving Model and Scaler ---")
try:
    joblib.dump(rf_model, 'm2.pkl')
    joblib.dump(scaler, 's2.pkl')
    joblib.dump(X.columns.tolist(), 'f2.pkl') # Save feature names for consistency
    print("Model, scaler, and feature names saved successfully.")
except Exception as e:
    print(f"Error saving model/scaler: {e}")




--- Saving Model and Scaler ---
Model, scaler, and feature names saved successfully.


In [79]:
import joblib

In [81]:
# --- Loading the trained model and scaler ---
print("\n--- Loading Model and Scaler for Prediction ---")
loaded_rf_model = None
loaded_scaler = None
loaded_features = None
try:
    loaded_rf_model = joblib.load('rf_model.pkl')
    loaded_scaler = joblib.load('scaler.pkl')
    loaded_features = joblib.load('model_features.pkl')
    print("Model, scaler, and feature names loaded successfully.")
except FileNotFoundError:
    print("Saved model or scaler not found. Please run the training section first.")
except Exception as e:
    print(f"Error loading model/scaler: {e}")



--- Loading Model and Scaler for Prediction ---
Model, scaler, and feature names loaded successfully.


In [ ]:
import pandas as pd
import numpy as np
import joblib

# --- Placeholder for loading the pre-trained model and scaler ---
# In a real-world scenario, you would have saved these objects after training.
try:
    loaded_rf_model = joblib.load('rf_model.pkl')
    loaded_scaler = joblib.load('scaler.pkl')
    loaded_features = joblib.load('model_features.pkl')
    print("Model, scaler, and feature columns loaded successfully.")
except FileNotFoundError:
    print("Error: Model or scaler files not found. Please ensure 'model.joblib', 'scaler.joblib', and 'features.joblib' are in the same directory.")
    loaded_rf_model, loaded_scaler, loaded_features = None, None, None

# --- Function to make predictions for new data ---
def predict_new_rent_from_input(model, scaler, original_df_columns, categorical_features, numerical_cols_present):
    """
    Prompts the user for house features and predicts the rental price.
    Ensures numerical inputs are strictly numbers for numerical fields
    and provides numbered options for categorical fields.
    """
    if model is None or scaler is None or original_df_columns is None:
        print("Model, scaler, or feature columns not loaded. Cannot make predictions.")
        return

    print("\n--- Enter House Details for Prediction ---")
    new_data_dict = {}

    # Helper function to get numerical input with validation
    def get_numerical_input(prompt):
        while True:
            try:
                value = int(input(prompt))
                return value
            except ValueError:
                print("Invalid input. Please enter a number.")

    # Helper function to get categorical input with numbered options
    def get_categorical_input(prompt, options_list):
        while True:
            print(prompt)
            for i, option in enumerate(options_list):
                print(f"{i+1}. {option}")
            try:
                choice = int(input("Enter the number corresponding to your choice: "))
                if 1 <= choice <= len(options_list):
                    return options_list[choice-1]
                else:
                    print("Invalid number. Please choose from the given options.")
            except ValueError:
                print("Invalid input. Please enter a number.")

    # Get user inputs for all features
    new_data_dict['Size_In_Sqft'] = get_numerical_input("Enter Size_In_Sqft: ")
    new_data_dict['Carpet_Area_Sqft'] = get_numerical_input("Enter Carpet_Area_Sqft: ")
    new_data_dict['Bedrooms'] = get_numerical_input("Enter Number of Bedrooms: ")
    new_data_dict['Bathrooms'] = get_numerical_input("Enter Number of Bathrooms: ")
    new_data_dict['Balcony'] = get_numerical_input("Enter Number of Balcony: ")
    new_data_dict['Number_Of_Amenities'] = get_numerical_input("Enter Number_Of_Amenities: ")
    # Note: 'Security_Deposite' is not in the numerical_cols_present list. This should be verified.
    new_data_dict['Security_Deposite'] = get_numerical_input("Enter Security_Deposite: ")
    new_data_dict['Floor_No'] = get_numerical_input("Enter Floor_No: ")
    new_data_dict['Total_floors_In_Building'] = get_numerical_input("Enter Total_floors_In_Building: ")
    new_data_dict['Road_Connectivity'] = get_numerical_input("Enter Road_Connectivity (1-10 scale): ")
    new_data_dict['gym'] = get_numerical_input("Enter 1 if gym is available, 0 otherwise: ")
    new_data_dict['gated_community'] = get_numerical_input("Enter 1 if gated_community is available, 0 otherwise: ")
    new_data_dict['intercom'] = get_numerical_input("Enter 1 if intercom is available, 0 otherwise: ")
    new_data_dict['lift'] = get_numerical_input("Enter 1 if lift is available, 0 otherwise: ")
    new_data_dict['pet_allowed'] = get_numerical_input("Enter 1 if pet_allowed is available, 0 otherwise: ")
    new_data_dict['pool'] = get_numerical_input("Enter 1 if pool is available, 0 otherwise: ")
    new_data_dict['security'] = get_numerical_input("Enter 1 if security is available, 0 otherwise: ")
    new_data_dict['water_supply'] = get_numerical_input("Enter 1 if water_supply is available, 0 otherwise: ")
    new_data_dict['wifi'] = get_numerical_input("Enter 1 if wifi is available, 0 otherwise: ")
    new_data_dict['gas_pipeline'] = get_numerical_input("Enter 1 if gas_pipeline is available, 0 otherwise: ")
    new_data_dict['sports_facility'] = get_numerical_input("Enter 1 if sports_facility is available, 0 otherwise: ")
    new_data_dict['kids_area'] = get_numerical_input("Enter 1 if kids_area is available, 0 otherwise: ")
    new_data_dict['power_backup'] = get_numerical_input("Enter 1 if power_backup is available, 0 otherwise: ")
    new_data_dict['Garden'] = get_numerical_input("Enter 1 if Garden is available, 0 otherwise: ")
    new_data_dict['Fire_Support'] = get_numerical_input("Enter 1 if Fire_Support is available, 0 otherwise: ")
    new_data_dict['Parking'] = get_numerical_input("Enter 1 if Parking is available, 0 otherwise: ")
    new_data_dict['ATM_Near_me'] = get_numerical_input("Enter 1 if ATM is Near_me, 0 otherwise: ")
    new_data_dict['Airport_Near_me'] = get_numerical_input("Enter 1 if Airport is Near_me, 0 otherwise: ")
    new_data_dict['Bus_Stop__Near_me'] = get_numerical_input("Enter 1 if Bus_Stop is Near_me, 0 otherwise: ")
    new_data_dict['Hospital_Near_me'] = get_numerical_input("Enter 1 if Hospital is Near_me, 0 otherwise: ")
    new_data_dict['Mall_Near_me'] = get_numerical_input("Enter 1 if Mall is Near_me, 0 otherwise: ")
    new_data_dict['Market_Near_me'] = get_numerical_input("Enter 1 if Market is Near_me, 0 otherwise: ")
    new_data_dict['Metro_Station_Near_me'] = get_numerical_input("Enter 1 if Metro_Station is Near_me, 0 otherwise: ")
    new_data_dict['Park_Near_me'] = get_numerical_input("Enter 1 if Park is Near_me, 0 otherwise: ")
    new_data_dict['School_Near_me'] = get_numerical_input("Enter 1 if School is Near_me, 0 otherwise: ")
    new_data_dict['Property_Age'] = get_numerical_input("Enter Property_Age: ")
    
    # Define options for categorical inputs based on common values in the dataset
    area_options = [
    'Hingna', 'Trimurti Nagar', 'Ashirwad Nagar', 'Beltarodi', 'Besa',
    'Bharatwada', 'Boriyapura', 'Chandrakiran Nagar', 'Dabha', 'Dhantoli',
    'Dharampeth', 'Dighori', 'Duttawadi', 'Gandhibagh', 'Ganeshpeth',
    'Godhni', 'Godhni', 'Gotal Panjri', 'Hingna', 'Hudkeswar', 'Itwari',
    'Jaitala', 'Jaripatka', 'Kalamna', 'Kalmeshwar', 'Khamla', 'Kharbi',
    'Koradi Colony', 'Kotewada', 'Mahal', 'Manewada', 'Manish Nagar',
    'Mankapur', 'Medical Square', 'MIHAN', 'Nandanwan',
    'Narendra Nagar Extension', 'Nari Village', 'Narsala', 'Omkar Nagar',
    'Parvati Nagar', 'Pratap Nagar', 'Ram Nagar', 'Rameshwari',
    'Reshim Bagh', 'Sadar', 'Sanmarga Nagar', 'Seminary Hills',
    'Shatabdi Square', 'Shatabdi Square', 'Sitabuldi', 'Somalwada',
    'Sonegaon', 'Teka Naka', 'Vayusena Nagar', 'Wanadongri',
    'Wardsman Nagar', 'Wathoda', 'Zingabai Takli'
    ]
    zone_options = ['East Zone', 'North Zone', 'South Zone', 'West Zone', 'Rural']
    furnishing_status_options = ['Fully Furnished', 'Semi Furnished', 'Unfurnished']
    recommended_for_options = ['Anyone', 'Bachelors', 'Family', 'Family and Bachelors', 'Family and Company']
    water_supply_options = ['Borewell', 'Both', 'Municipal']

    new_data_dict['City'] = 'Nagpur' # Assuming a single city, as per original data context
    new_data_dict['Area'] = get_categorical_input("Select Area:", area_options)
    new_data_dict['Zone'] = get_categorical_input("Select Zone:", zone_options)
    new_data_dict['Frurnishing_Status'] = get_categorical_input("Select Furnishing Status:", furnishing_status_options)
    new_data_dict['Recomened for'] = get_categorical_input("Select Recommended For:", recommended_for_options)
    new_data_dict['Muncipla Water Or Bore Water'] = get_categorical_input("Select Water Supply:", water_supply_options)
    
    # Missing categorical inputs in the user prompt section
    # Let's add them based on the categorical_features list
    type_of_society_options = ['Gated','Non-Gated','Township']
    new_data_dict['Type of Society'] = get_categorical_input("Select Type of Society:", type_of_society_options)
    
    room_options = ['1 RK', '1 BHK', '2 BHK', '3 BHK', '4 BHK', '5+ BHK']
    new_data_dict['Room'] = get_categorical_input("Select Room type:", room_options)
    
    type_options = ['Flat','Studio Apartment','Independent House','Independent Builder Floor','Villa','Duplex']
    new_data_dict['Type'] = get_categorical_input("Select Property Type:", type_options)
    
    brokerage_options = ['No Brokerage', 'With Brokerage']
    new_data_dict['Brokerage'] = get_categorical_input("Select Brokerage:", brokerage_options)
    
    maintenance_charge_options = ['Maintenance Not Included', 'Maintenance Included']
    new_data_dict['Maintenance_Charge'] = get_categorical_input("Select Maintenance Charge:", maintenance_charge_options)


    # Create a DataFrame from the new data dictionary
    new_df = pd.DataFrame([new_data_dict])
    
    # Apply one-hot encoding to the new DataFrame
    for feature in categorical_features:
        if feature in new_df.columns:
            # Handle the case where the input value for a category might be new
            try:
                new_df = pd.get_dummies(new_df, columns=[feature], drop_first=True)
            except ValueError as e:
                print(f"Warning: Could not one-hot encode '{feature}'. The value provided might not be in the training data categories. Error: {e}")
                # Create a placeholder column for the feature to prevent errors
                new_df[feature] = new_data_dict[feature]

    # Align columns with the training data (important for one-hot encoding)
    missing_cols = set(original_df_columns) - set(new_df.columns)
    for c in missing_cols:
        new_df[c] = 0
    new_df = new_df[original_df_columns] # Ensure order is the same

    # Scale numerical features
    new_df[numerical_cols_present] = scaler.transform(new_df[numerical_cols_present])

    # Make prediction
    try:
        log_predicted_rent = model.predict(new_df)[0]
        predicted_rent = np.expm1(log_predicted_rent) # Inverse transform
    except Exception as e:
        print(f"Prediction failed. Check your inputs and the model. Error: {e}")
        return

    print(f"\nPredicted Rent: Rs{predicted_rent:.2f}")

    # --- Price Classification ---
    FAIR_PRICE_TOLERANCE = 0.07 # 7%
    lower_bound = predicted_rent * (1 - FAIR_PRICE_TOLERANCE)
    upper_bound = predicted_rent * (1 + FAIR_PRICE_TOLERANCE)
    print(f"A fair price for this property would typically be between Rs{lower_bound:.2f} and Rs{upper_bound:.2f}.")

    listed_price = get_numerical_input("Enter the listed price of the property for comparison (e.g., 25000): ")
    
    if listed_price < lower_bound:
        print("This property appears to be **Underpriced**!")
    elif listed_price > upper_bound:
        print("This property appears to be **Overpriced**!")
    else:
        print("This property appears to be **Fairly Priced**.")


# --- Main execution block for user input ---
if __name__ == "__main__":
    # Assuming these variables are populated from the training script
    categorical_features = ['City', 'Area', 'Zone', 'Frurnishing_Status', 'Brokerage', 'Maintenance_Charge', 'Recomened for', 'Muncipla Water Or Bore Water', 'Type of Society', 'Room', 'Type']
    numerical_cols = ['Size_In_Sqft', 'Carpet_Area_Sqft', 'Bedrooms', 'Bathrooms', 'Balcony',
'Number_Of_Amenities', 'Floor_No', 'Total_floors_In_Building',
'Road_Connectivity', 'gated_community', 'gym', 'intercom', 'lift', 'pet_allowed', 'pool',
'security', 'water_supply', 'wifi', 'gas_pipeline', 'sports_facility', 'kids_area',
'power_backup', 'Garden', 'Fire_Support', 'Parking', 'ATM_Near_me', 'Airport_Near_me',
'Bus_Stop__Near_me', 'Hospital_Near_me', 'Mall_Near_me', 'Market_Near_me',
'Metro_Station_Near_me', 'Park_Near_me', 'School_Near_me', 'Property_Age']

    if loaded_rf_model and loaded_scaler and loaded_features:
        # Filter numerical_cols to only include columns actually present in the loaded features
        numerical_cols_present = [col for col in numerical_cols if col in loaded_features]
        while True:
            predict_new_rent_from_input(loaded_rf_model, loaded_scaler, loaded_features, categorical_features, numerical_cols_present)
            another_prediction = input("\nDo you want to predict another rent? (yes/no): ").lower()
            if another_prediction != 'yes':
                break
    else:
        print("\nCannot start interactive prediction. Model, scaler, or feature list not loaded.")

Model, scaler, and feature columns loaded successfully.

--- Enter House Details for Prediction ---
